## SILVER TO GOLD LAYER

### Gold Layer - Holding


In [1]:
# Importing Common Utility Function
import pandas as pd
from StockETL import GlobalPath, Portfolio

In [2]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

In [3]:
# Instantiate GlobalPath
tradehistory_silver_file_path = GlobalPath(
    "DATA/SILVER/TradeHistory/TradeHistory_data.csv"
)
stockprice_silver_file_path = GlobalPath("DATA/SILVER/StockPrice/StockPrice_data.csv")
symbol_silver_file_path = GlobalPath("DATA/SILVER/Symbol/Symbol_data.csv")

current_holding_records_file_path = GlobalPath(
    "DATA/GOLD/Holding/CurrentHolding_data.csv"
)
current_holding_gold_schema_file_path = GlobalPath(
    "CONFIG/DATA_CONTRACTS/GOLD/CurrentHolding.json"
)

holding_gold_file_path = GlobalPath("DATA/GOLD/Holding/Holding_data.csv")
holding_gold_schema_file_path = GlobalPath("CONFIG/DATA_CONTRACTS/GOLD/Holding.json")

holding_source_layer_path = GlobalPath("DATA/SOURCE/Holding/Holding_data.csv")
holding_source_schema_file_path = GlobalPath(
    "CONFIG/DATA_CONTRACTS/SOURCE/Holding.json"
)

### Data Processing

- Read and sort trade history data.
- Apply portfolio trade logic.


In [ ]:
# Read the CSV file
df_trade_history = pd.read_csv(tradehistory_silver_file_path)

# Filter the DataFrame to include only rows where the segment is 'EQ' (Equity) or 'MF' (Mutual Funds)
df_trade_history = df_trade_history[df_trade_history["segment"].isin(["EQ", "MF"])]

# Convert 'datetime' to datetime type
df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])

# Sort the DataFrame by 'datetime'
df_trade_history = df_trade_history.sort_values(by="datetime")

print(f"Read SILVER Layer trade history data from => {tradehistory_silver_file_path}")

### Portfolio Logic Application

- Instantiate Portfolio and apply trade logic.
- Handle expired stocks


In [5]:
# Apply the trade logic to each row of the DataFrame
# Instantiate the Portfolio object
portfolio = Portfolio()
# .astype(str)
for record in df_trade_history.to_dict(orient="records"):
    portfolio.trade(record)

portfolio.check_expired_stocks()

In [ ]:
# Create a DataFrame from the processed data
df_currentholding = pd.DataFrame(portfolio.get_current_holding())

# Align Datafame with DataContract
df_currentholding = align_with_datacontract(
    df_currentholding, current_holding_gold_schema_file_path
)

# Save the final DataFrame to a CSV file
df_currentholding.to_csv(current_holding_records_file_path, index=None)

print("GOLD Layer CSV file for Current Holdings successfully created at =>")
print(current_holding_records_file_path)

# Display the DataFrame debugrmation
df_currentholding.info()

### Final Processing and Export

- Select and sort relevant columns.
- Save the processed data as a CSV file in the Gold layer.


In [7]:
# Create a DataFrame from the processed data
df_holding = pd.DataFrame(portfolio.get_holding_history())

# Extract date from datetime
df_holding["date"] = df_holding["datetime"].dt.date

# Find the index of the maximum datetime for each scrip_name and date
idx = df_holding.groupby(["scrip_name", "date"])["datetime"].idxmax()

# Use the indices to filter the original DataFrame
df_holding = df_holding.loc[idx].reset_index(drop=True)

In [8]:
# Create a function to expand the date range for each stock
def expand_dates(stock_df):
    """
    Expands the date range for each stock to include all dates from the
    minimum date to the current date and forward fills the missing values.

    Parameters:
    stock_df (pd.DataFrame): DataFrame containing stock data for a single stock.

    Returns:
    pd.DataFrame: DataFrame with expanded date range and forward-filled values.
    """
    # Find the minimum date for this stock
    min_date = stock_df["date"].min()

    # Create a date range from the minimum date to today
    date_range = pd.date_range(start=min_date, end=pd.to_datetime("today"))

    # Reindex the stock_df to include the full date range
    stock_df = stock_df.set_index("date").reindex(date_range)

    # Forward fill the values to fill missing dates
    stock_df = stock_df.ffill().reset_index()

    # Rename the 'index' column to 'date'
    stock_df = stock_df.rename(columns={"index": "date"})

    return stock_df


# Apply the expand_dates function to each stock group
df_holding = (
    df_holding.groupby("scrip_name")
    .apply(expand_dates, include_groups=False)
    .reset_index()
)

In [ ]:
# Load stock prices data from the specified CSV file path
df_stockprice = pd.read_csv(stockprice_silver_file_path)
print(f"Loaded SILVER Layer stock price data from => {stockprice_silver_file_path}")

# Convert the 'date' column in stock prices to datetime type for consistency
df_stockprice["date"] = pd.to_datetime(df_stockprice["date"])
df_stockprice.info()

In [10]:
# Merge the expanded holding data with the stock price data
df_holding = pd.merge(
    df_holding,
    df_stockprice,
    on=["date", "symbol"],
    how="left",
)

# Calculate the OHLC value of holding
col_names = ["open", "high", "low", "close"]
for col_name in col_names:
    df_holding[f"{col_name}_price"] = df_holding[col_name]
    df_holding[f"{col_name}_amount"] = (
        df_holding[col_name] * df_holding["holding_quantity"]
    )

# Forward fill the values to fill missing dates
df_holding = df_holding.ffill()

# Filter out rows with zero holding quantity
df_holding = df_holding[df_holding["holding_quantity"] != 0]

In [ ]:
# Reset index to ensure it starts from 0
df_holding = df_holding.reset_index(drop=True)

# Align Datafame with DataContract
df_holding = align_with_datacontract(df_holding, holding_gold_schema_file_path)

# Save the final processed DataFrame to a new CSV file
df_holding.to_csv(holding_gold_file_path, index=None)

print(
    f"GOLD Layer CSV file for Holding successfully created at: {holding_gold_file_path}"
)

# Display debugrmation about the final DataFrame
df_holding.info()

In [ ]:
# Replace scrip code with scrip_name name
df_symbol = pd.read_csv(symbol_silver_file_path)
print(f"Loaded data from => {symbol_silver_file_path}")

# Calculate the min and max dates for each stock
df_holding = (
    df_holding.groupby(["segment", "exchange", "symbol"])
    .agg(min_date=("date", "min"), max_date=("date", "max"))
    .reset_index()
)

# Merge df_HoldingHistory with df_Symbol on the matching columns
df_holding = df_holding.merge(
    df_symbol[["symbol", "isin"]],
    left_on="symbol",
    right_on="symbol",
    how="left",
)

# Align Datafame with DataContract
df_holding = align_with_datacontract(df_holding, holding_source_schema_file_path)

# Save the final processed DataFrame to a new CSV file
df_holding.to_csv(holding_source_layer_path, index=None)

print(
    f"GOLD Layer CSV file for Holding successfully created at: {holding_source_layer_path}"
)

# Display debugrmation about the final DataFrame
df_holding.info()